# Navier Stokes - blood flow simulation

## Formulation

In this example we present a bloodflow simulation in a 3D geometry and we demonstrate how to prescribe a time dependent expression as a Dirichlet boundary condition.
![Mesh](fig/aneurysm.png "Mesh")

## Implementation
Again, we need to import the neccessary packages.

In [24]:
import dolfin as df
import numpy as np
from time import time

Next, we read the mesh from the file and create the boundary markings since ```.xml``` files do not include them.

In [25]:
mesh = df.Mesh("bifurcation.xml")
dim = mesh.topology().dim()
bndry = df.MeshFunction("size_t", mesh, dim-1, 0)
mesh.init()

points = [np.array([-0.9, 0.0, 0.0]), np.array([0.6, 0.0, 0.6]), np.array([0.6, 0.0, -0.6])]
normals = [np.array([-1.0, 0.0, 0.0]), np.array([np.sqrt(2), 0.0, np.sqrt(2)]), np.array([np.sqrt(2), 0.0, -np.sqrt(2)])]
for f in df.facets(mesh):
    bndry[f] = 0
    if f.exterior():
        bndry[f] = 1
        x = f.midpoint().array()[:dim]
        for mark, point, normal in zip([2,3,4], points, normals):
            if abs(np.dot(x-point, normal)) <= 0.001:
                bndry[f] = mark

Now we define the function spaces for MINI element.

In [26]:
U = df.FiniteElement("CG", mesh.ufl_cell(), 1)
B = df.FiniteElement("Bubble", mesh.ufl_cell(), dim + 1)
P = df.FiniteElement("CG", mesh.ufl_cell(), 1)
V = df.VectorElement(df.NodalEnrichedElement(U, B))
W = df.FunctionSpace(mesh, df.MixedElement([V, P]))

Let us define some parameters and constants. Here you can see an approximate values of density and viscosity of blood.

In [27]:
mu = df.Constant(0.04) # Pa*s
rho = df.Constant(1.05) # g/cm^3
theta = df.Constant(0.5)
dt = 0.01
t_end = 0.35
t = 0.0

Then we can define the boundary conditions as no slip on the wall and parabolic expression at the inlet. Notice that the average inflow velocity is now prescribed using a function of time. 

In [28]:
zero_vector = df.Constant((0.0, 0.0, 0.0))
bc_walls = df.DirichletBC(W.sub(0), zero_vector, bndry, 1)

velocity = lambda t: 0.6*max((t*(0.3-t))/pow(0.15,2), 0.0)
inlet_expression = df.Expression(("v * 2.0 * (pow(R,2) - pow(x[0]+0.9,2) - pow(x[1],2) - pow(x[2],2)) / pow(R,2)", "0.0", "0.0"), R=0.15, v=velocity(t), degree=2)
bc_in = df.DirichletBC(W.sub(0), inlet_expression, bndry, 2)

bcs = [bc_in, bc_walls]

Then we can define the variational problem the same way as in the previous examples.

In [29]:
v, q = df.TestFunctions(W)
w = df.Function(W)
w0 = df.Function(W)
u, p = df.split(w)
u0, p0 = df.split(w0)

a = lambda u, v: rho*df.inner(df.grad(u)*u, v)*df.dx + mu*df.inner(df.grad(u), df.grad(v))*df.dx
b = lambda q, v: q*df.div(v)*df.dx

F1 = a(u, v) - b(p, v) - b(q, u)
F0 = a(u0, v) - b(p, v) - b(q, u)

# time derivative
F = df.Constant(1.0/dt)*rho*df.inner((u-u0), v)*df.dx + theta*F1 + (1-theta)*F0

# Jacobi matrix
J = df.derivative(F, w)

Setting up nonlinear variational solver.

In [30]:
problem = df.NonlinearVariationalProblem(F, w, bcs, J)
solver = df.NonlinearVariationalSolver(problem)
solver.parameters['newton_solver']['linear_solver'] = 'mumps'
solver.parameters['newton_solver']['absolute_tolerance'] = 1e-8
solver.parameters['newton_solver']['relative_tolerance'] = 1e-6

Setup files to save results for viewing in Paraview.

In [31]:
ufile = df.XDMFFile("results/u.xdmf")
pfile = df.XDMFFile("results/p.xdmf")
ufile.parameters["flush_output"] = True
pfile.parameters["flush_output"] = True

Again, we go over all the timesteps and solve for velocity and pressure at each of them.

In [ ]:
tick = time()
t = 0.0
(u, p) = w.split(True)
u.rename("v", "velocity")
p.rename("p", "pressure")
df.assign(u, w.sub(0))
df.assign(p, w.sub(1))
ufile.write(u, t)
pfile.write(p, t)
while t < t_end:
    w0.assign(w)
    t += dt
    print("t={:.2f}s".format(t))
    inlet_expression.v = velocity(t)
    solver.solve()
    df.assign(u, w.sub(0))
    df.assign(p, w.sub(1))
    ufile.write(u, t)
    pfile.write(p, t)
print("ellapsed = ", time() - tick, "s")

## Complete Code

Run the code using the command ```python3 NavierStokes3D.py```.

In [1]:
import dolfin as df
import numpy as np
from time import time

mesh = df.Mesh("bifurcation.xml")
dim = mesh.topology().dim()
bndry = df.MeshFunction("size_t", mesh, dim-1, 0)
mesh.init()

points = [np.array([-0.9, 0.0, 0.0]), np.array([0.6, 0.0, 0.6]), np.array([0.6, 0.0, -0.6])]
normals = [np.array([-1.0, 0.0, 0.0]), np.array([np.sqrt(2), 0.0, np.sqrt(2)]), np.array([np.sqrt(2), 0.0, -np.sqrt(2)])]
for f in df.facets(mesh):
    bndry[f] = 0
    if f.exterior():
        bndry[f] = 1
        x = f.midpoint().array()[:dim]
        for mark, point, normal in zip([2,3,4], points, normals):
            if abs(np.dot(x-point, normal)) <= 0.001:
                bndry[f] = mark

U = df.FiniteElement("CG", mesh.ufl_cell(), 1)
B = df.FiniteElement("Bubble", mesh.ufl_cell(), dim + 1)
P = df.FiniteElement("CG", mesh.ufl_cell(), 1)
V = df.VectorElement(df.NodalEnrichedElement(U, B))
W = df.FunctionSpace(mesh, df.MixedElement([V, P]))

mu = df.Constant(0.04) # Pa*s
rho = df.Constant(1.05) # g/cm^3
theta = df.Constant(0.5)
dt = 0.01
t_end = 0.35
t = 0.0

zero_vector = df.Constant((0.0, 0.0, 0.0))
bc_walls = df.DirichletBC(W.sub(0), zero_vector, bndry, 1)

velocity = lambda t: 0.6*max((t*(0.3-t))/pow(0.15,2), 0.0)
inlet_expression = df.Expression(("v * 2.0 * (pow(R,2) - pow(x[0]+0.9,2) - pow(x[1],2) - pow(x[2],2)) / pow(R,2)", "0.0", "0.0"), R=0.15, v=velocity(t), degree=2)
bc_in = df.DirichletBC(W.sub(0), inlet_expression, bndry, 2)

bcs = [bc_in, bc_walls]

v, q = df.TestFunctions(W)
w = df.Function(W)
w0 = df.Function(W)
u, p = df.split(w)
u0, p0 = df.split(w0)

a = lambda u, v: rho*df.inner(df.grad(u)*u, v)*df.dx + mu*df.inner(df.grad(u), df.grad(v))*df.dx
b = lambda q, v: q*df.div(v)*df.dx

F1 = a(u, v) - b(p, v) - b(q, u)
F0 = a(u0, v) - b(p, v) - b(q, u)

# time derivative
F = df.Constant(1.0/dt)*rho*df.inner((u-u0), v)*df.dx + theta*F1 + (1-theta)*F0

# Jacobi matrix
J = df.derivative(F, w)

problem = df.NonlinearVariationalProblem(F, w, bcs, J)
solver = df.NonlinearVariationalSolver(problem)
solver.parameters['newton_solver']['linear_solver'] = 'mumps'
solver.parameters['newton_solver']['absolute_tolerance'] = 1e-8
solver.parameters['newton_solver']['relative_tolerance'] = 1e-6

ufile = df.XDMFFile("results/u.xdmf")
pfile = df.XDMFFile("results/p.xdmf")
ufile.parameters["flush_output"] = True
pfile.parameters["flush_output"] = True

tick = time()
t = 0.0
(u, p) = w.split(True)
u.rename("v", "velocity")
p.rename("p", "pressure")
df.assign(u, w.sub(0))
df.assign(p, w.sub(1))
ufile.write(u, t)
pfile.write(p, t)
while t < t_end:
    w0.assign(w)
    t += dt
    print("t={:.2f}s".format(t))
    inlet_expression.v = velocity(t)
    solver.solve()
    df.assign(u, w.sub(0))
    df.assign(p, w.sub(1))
    ufile.write(u, t)
    pfile.write(p, t)
print("ellapsed = ", time() - tick, "s")

t=0.01s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.438e+01 (tol = 1.000e-08) r (rel) = 1.000e+00 (tol = 1.000e-06)
  Newton iteration 1: r (abs) = 5.593e-06 (tol = 1.000e-08) r (rel) = 3.891e-07 (tol = 1.000e-06)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
t=0.02s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.338e+01 (tol = 1.000e-08) r (rel) = 1.000e+00 (tol = 1.000e-06)
  Newton iteration 1: r (abs) = 4.335e-06 (tol = 1.000e-08) r (rel) = 3.239e-07 (tol = 1.000e-06)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
t=0.03s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.239e+01 (tol = 1.000e-08) r (rel) = 1.000e+00 (tol = 1.000e-06)
  Newton iteration 1: r (abs) = 3.469e-06 (tol = 1.000e-08) r (rel) = 2.799e-07 (tol = 1.000e-06)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
t=0.04s
Solving nonlinear variational problem.
  New

t=0.28s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.239e+01 (tol = 1.000e-08) r (rel) = 1.000e+00 (tol = 1.000e-06)
  Newton iteration 1: r (abs) = 3.473e-06 (tol = 1.000e-08) r (rel) = 2.802e-07 (tol = 1.000e-06)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
t=0.29s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.338e+01 (tol = 1.000e-08) r (rel) = 1.000e+00 (tol = 1.000e-06)
  Newton iteration 1: r (abs) = 4.073e-06 (tol = 1.000e-08) r (rel) = 3.043e-07 (tol = 1.000e-06)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
t=0.30s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.438e+01 (tol = 1.000e-08) r (rel) = 1.000e+00 (tol = 1.000e-06)
  Newton iteration 1: r (abs) = 4.701e-06 (tol = 1.000e-08) r (rel) = 3.270e-07 (tol = 1.000e-06)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
t=0.31s
Solving nonlinear variational problem.
  New